In [4]:
def toggle_warning():
    HTML('''<script>
    code_show_err=false; 
    function code_toggle_err() {
     if (code_show_err){
     $('div.output_stderr').hide();
     } else {
     $('div.output_stderr').show();
     }
     code_show_err = !code_show_err
    } 
    $( document ).ready(code_toggle_err);
    </script>
    To toggle on/off output_stderr, click <a href="javascript:code_toggle_err()">here</a>.''')

In [5]:
import tensorflow as tf
from IPython.display import HTML

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [6]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
toggle_warning()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [7]:
#helper functions
#init weights

def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(init_random_dist)

In [8]:
#init bias
def init_bias(shape):
    init_bias_vals = tf.constant(0.1,shape=shape)
    return tf.Variable(init_bias_vals)

In [9]:
#conv2d
def conv2d(x,W):
    # x --> input tensor -> shape = [batch,H,W,Channels]
    #batch -> batch of images ; H/W = height/width of images
    #Channels -> greyscale - > 1; RGB - > Multiple channels
    # W --> kernel -> shape = [filter H, filter W, Channels IN, Channels OUT]
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')
#padding means padding image boundary with Zeros
#strides mean how many pixels to jump when filtering

In [10]:
#pooling
def max_pool_2by2(x):
    # x --> input tensor -> shape = [batch,H,W,Channels]
    #batch -> batch of images ; H/W = height/width of images
    #Channels -> greyscale - > 1; RGB - > Multiple channels
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [20]:
#convolution layer
def conv_layer(input_x,shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x,W)+b)
#activation function takes input x, a weight and adds bias

In [21]:
#normal/fully connected layer
def normal_full_layer(input_layer,size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size,size])
    b = init_bias([size])
    return tf.matmul(input_layer,W) + b

In [13]:
#placeholders
x = tf.placeholder(tf.float32,shape=[None,784])

In [14]:
y_true = tf.placeholder(tf.float32,shape=[None,10])

In [15]:
#layers

#reshape flattened image from 784 pixels to 28x28 pixels
#last 1 define channel which means grayscale
x_image = tf.reshape(x,[-1,28,28,1])

In [26]:
#5x5 patch of image, 1 channel -> grayscale
#32 -> output channel
convo_1 = conv_layer(x_image,shape=[5,5,1,32])

convo1_pooling = max_pool_2by2(convo_1)

In [27]:
#shape -> 5x5 img patch
#input of this layer is 32 and output is 64 features
convo_2 = conv_layer(convo1_pooling,shape=[5,5,32,64])
convo2_pooling = max_pool_2by2(convo_2)

In [28]:
#flatten our layer to use for fully connected layer
convo_2_flat = tf.reshape(convo2_pooling,[-1,7*7*64])
fully_layer_1 = tf.nn.relu(normal_full_layer(convo_2_flat,1024))

In [30]:
#dropout
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(fully_layer_1,rate=1-hold_prob)

In [31]:
y_pred = normal_full_layer(full_one_dropout,10)

In [33]:
#loss function -> cross entropy
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_true,logits=y_pred))

In [34]:
#optmizier -> ADAM
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

In [36]:
init = tf.global_variables_initializer()

In [39]:
steps = 500

with tf.Session() as sess:
    sess.run(init)
    
    for i in range(steps):
        batch_x , batch_y = mnist.train.next_batch(50)
        
        sess.run(train,feed_dict={x:batch_x,y_true:batch_y,hold_prob:0.5})
        
        if i % 100 == 0 : #for every hundred steps, do
            print("ON STEP : {}".format(i))
            print("Accuracy: ")
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
            acc = tf.reduce_mean(tf.cast(matches,tf.float32))
            
            print(sess.run(acc,feed_dict={x:mnist.test.images,y_true:mnist.test.labels,hold_prob:1.0}))
            print('\n')

ON STEP : 0
Accuracy: 
0.1551


ON STEP : 100
Accuracy: 
0.9431


ON STEP : 200
Accuracy: 
0.9649


ON STEP : 300
Accuracy: 
0.9722


ON STEP : 400
Accuracy: 
0.9683




As you can see, the accuracy is much better than the softmax CNN and the basic MATMUL 4_layer NN. This can be improved to 99% if the number of steps is increased.